<a href="https://colab.research.google.com/github/DiyaJ2002/DiyaJain/blob/main/SentimentsAnalysisUsingTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Twitter Sentimeny Analysis Using Python

In [ ]:
#import the libraries needed
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Twitter API keys and Tokens
Access_token="1546858586437189634-2MmTjCZWYMVUDhRfTrEmYoHk0UUvKp"
Access_token_secret="glc9BrPiZXwrNyNA7CI4OZ4NfHIcegSQmwdgWnD39QPp3"
Consumer_key="KYYLiUqWCkCBGSnvF5RKR6XoH"
Consumer_secret="IrQEA0xApWF4NmPjCFDPmmrLLAG3pBKT1fy4jKH1r5Ne67xnNA"

In [ ]:
#Creating OAuthHandler instance and make it equipped with access token
auth = tweepy.OAuthHandler(Consumer_key, Consumer_secret)
auth.set_access_token(Access_token, Access_token_secret)
api= tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
#Verifying Our credentials using exception handling
try: 
  api.verify_credentials()
  print('Success')
except:
  print('failed')

In [ ]:
#Taking number of tweets we want to analyze as input
num= int(input("Enter the number of tweets you want to analyze:"))

In [ ]:
#Taking hashtag or keyword as an input
searchTerm = input("Enter Keyword/Tag to search about: ")
search_query = searchTerm

In [ ]:
# get tweets from the API
tweets = tweepy.Cursor(api.search,q=search_query,lang="en",since="2020-09-16").items(num)
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

In [ ]:
#Fetching the status of tweets using api.get_status() method of api class in tweepy module
text = api.get_status(id=tweet.id, tweet_mode='extended').full_text

In [ ]:
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name,'user_description': tweet.user.description,'date': tweet.created_at,'text': text, 'hashtags': [hashtags if hashtags else None],}))
    tweets_df = tweets_df.reset_index(drop=True)
# show the dataframe
tweets_df.head()

In [ ]:
#Method to remove hashtags, retweets, mentions and urls from the tweets
def cleanText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '',text) #r tells that it is a raw string
    text = re.sub(r'#','',text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    return text
tweets_df['text']= tweets_df['text'].apply(cleanText)
tweets_df

In [ ]:
#Calculating subjectivity of tweets using TextBlob library 
def getSub(text):
  return TextBlob(text).sentiment.subjectivity

#Calculating polarity of tweets using TextBlob library 
def getPolarity(text):
   return TextBlob(text).sentiment.polarity

tweets_df['Subjectivity']=tweets_df['text'].apply(getSub)
tweets_df['Polarity']=tweets_df['text'].apply(getPolarity)
tweets_df

In [ ]:
#representing the data in worldcloud form
allWords = ' '.join( [twts for twts in tweets_df['text']])
wordCloud = WordCloud(width=1000,height=700, random_state=21, max_font_size=119).generate(allWords)

plt.imshow(wordCloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
#defining as positive, negative and neutral
def getAnalysis(score):
  if score<0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

tweets_df['Analysis']=tweets_df['Polarity'].apply(getAnalysis)

tweets_df

In [ ]:
#positive tweets
j=1
sortedDF = tweets_df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if( sortedDF['Analysis'][i]=='Positive'):
     print(str(j) + ')' +sortedDF['text'][i])
     print()
     j=j+1

In [ ]:
#Negative tweets
j=1
sortedDF = tweets_df.sort_values(by=['Polarity'], ascending='False')
for i in range(0,sortedDF.shape[0]):
  if(sortedDF['Analysis'][i]=='Negative'):
    print(str(j)+')'+sortedDF['text'][i])
    print()
    j=j+1

In [ ]:
#Scatter plot represenation of data
plt.figure(figsize=(8,6))
for i in range(0,tweets_df.shape[0]):
  plt.scatter(tweets_df['Polarity'][i],tweets_df['Subjectivity'][i], color='Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
#percentage of positive tweets
ptweets = tweets_df[tweets_df.Analysis=='Positive']
ptweets = ptweets['text']

round((ptweets.shape[0]/tweets_df.shape[0])*100, 1)

In [ ]:
#percentage of negative
ntweets = tweets_df[tweets_df.Analysis=='Negative']
ntweets = ntweets['text']

round((ntweets.shape[0]/tweets_df.shape[0])*100,1)

In [ ]:
#Bar graph representation of data
tweets_df['Analysis'].value_counts()

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
tweets_df['Analysis'].value_counts().plot(kind='bar')
plt.show()